## paso 0

In [3]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
import shap
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

# Configuración para mostrar todas las columnas en pandas
pd.set_option('display.max_columns', None)

print("Librerías cargadas correctamente.")

Librerías cargadas correctamente.


In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib

# 1. Variables de conexión proporcionadas
usuario = os.environ.get("SQL_USER","")
clave = os.environ.get("SQL_PASSWORD","")  # Nota: usamos comillas simples afuera para proteger las dobles
servidor = os.environ.get("SQL_SERVER","")
base_datos = os.environ.get("SQL_DB","")

# 2. Configuración de la cadena de conexión (Maneja caracteres especiales)
params = urllib.parse.quote_plus(
    f'DRIVER={{SQL Server}};'
    f'SERVER={servidor};'
    f'DATABASE={base_datos};'
    f'UID={usuario};'
    f'PWD={clave}'
)

# Crear el motor de conexión
# Si el driver estándar no funciona, prueba cambiando {SQL Server} por {ODBC Driver 17 for SQL Server}
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# 3. Tu consulta SQL completa
query = """
SELECT*FROM [CUN_REPOSITORIO].[COE].[df_final_GRU]
"""

print(f"Conectando al servidor {servidor}...")

try:
    # 4. Ejecución del proceso
    # pandas leerá los datos usando la conexión establecida
    df = pd.read_sql(query, engine)

    #print(f"✅ ¡Éxito! El archivo '{nombre_archivo}' ha sido generado.")
    print(f"📊 Total de registros exportados: {len(df)}")

except Exception as e:
    print(f"❌ Ocurrió un error: {e}")

Conectando al servidor 172.16.1.33...
📊 Total de registros exportados: 1750326


In [6]:
import os
df_o = df.copy()

In [82]:
import os
df = df_o

In [83]:
import os
df.head(1)

,Identificacion,Periodo,DescRF_Status,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_SEMESTRE_MEN,DescRF_Genero,DescRF_Unidad,DescRF_Jornada,DescRF_Periodo_Ult_Pago,DescRF_Sede,DescRF_Regional,DescRF_Programa,DescRF_Acceso_Moodle,DescRF_tipo_estudio,Id_Alumn_Programa,DescRF_Tipo_Alumno,DescRF_Nuevo,DescRF_orden,DescRF_Fecha_Liquidacion,DescRF_Fecha_Nacimiento,DescRF_ultimo_curso,DescAM_MATERIAS_INSCRITAS,DescAM_MATERIAS_APROBADAS,DescAM_Porcentaje_aprobacion,EEpromedio,EE_UNIDADNEGOCIO,EE_DEPARTAMENTO_REGIONAL,EE_OTRA_DISCAPACIDAD,EE_SISTEMA_SALUD,EE_ESTRATO_ACTUALIZADO,EE_EMPRESA,COD_MATERIA,DEF_HISTORIA,NOT_PERIODO,ESTADO_PAGO,EstadoMoodle,Ultimo_Acc_Mood,Correo,Horario,VALOR_FINANCIACION,RECIBOS_CAJA,ICETEX,FINCOMERCIO,CREDITY,JAC,CONVENIOV,Total_Tickets,Edad,Trabaja,diferencia_dias_moodle_u,NA_Calificaciones,NA_RECIBOS,NA_ICETEX,NA_FINCOMERCIO,NA_CREDITY,NA_Total_Tickets
0,1000004469,24V02,1,NUEVO,Virtual,8,PROFESIONAL,1,M,PROFESIONAL EN INGENIERIA DE SISTEMAS,JORNADA ÚNICA,24I02,BOGOTA CENTRO,Regional Bogotá,INGENIERIA DE SISTEMAS,Sin Moodle,PREGRADO,1152303,Nuevo,NUEVO,2451252,2024-03-18,2003-04-11,SI,6,3,50.0,None,None,None,None,None,None,None,TIE05,4.5,4.5,NO,ACTIVO,NaT,NO,SI,NaN,1539000.0,0.0,0.0,0.0,0.0,0.0,0.0,22,NO TRABAJA,NaN,False,False,False,False,False,True


In [84]:
import os
df.DescRF_Unidad.value_counts(dropna=False).head()

DescRF_Unidad
TECNICA PROFESIONAL EN PROCESOS ADMINISTRATIVOS             374062
TECNICA PROFESIONAL EN SOPORTE DE SISTEMAS E INFORMATICA    182620
TECNICA PROFESIONAL EN PROCESOS CONTABLES                   141114
TECNICA PROFESIONAL EN REGISTRO DE PROCESOS PRODUCTIVOS     101342
PROFESIONAL EN INGENIERIA DE SISTEMAS                        90421
Name: count, dtype: int64

In [85]:
import os
df.DescRF_Programa.value_counts(dropna=False).head()

DescRF_Programa
ADMINISTRACION DE EMPRESAS    487448
INGENIERIA DE SISTEMAS        330648
CONTADURIA PUBLICA            262761
INGENIERIA INDUSTRIAL         157309
DISENO GRAFICO                 76443
Name: count, dtype: int64

In [86]:
import os
df_final = df.drop(['Periodo','DescRF_SEMESTRE_MEN','Id_Alumn_Programa',
'DescRF_orden','DescRF_Fecha_Liquidacion','DescRF_ultimo_curso','EE_DEPARTAMENTO_REGIONAL',
'COD_MATERIA','DEF_HISTORIA','Ultimo_Acc_Mood','Correo','RECIBOS_CAJA','Edad','DescRF_Acceso_Moodle','EE_UNIDADNEGOCIO',
                   'DescRF_Unidad'],axis=1)

## paso 1

In [87]:
import os
# 1. Cargar la base de datos
# Asegúrate de cambiar la ruta si es necesario
#ruta_archivo = r"data/archivo.xlsx" 
#df_final = pd.read_excel(ruta_archivo)

# 2. Limpieza básica del ID
df_final['Identificacion'] = df_final['Identificacion'].astype(str).str.strip()

# 3. Definición de todas las variables posibles (Tu lista completa)
todas_las_variables = ['Identificacion','DescRF_Status','DescRF_Tipo_estado_alumno',
'DescRF_Modalidad','DescRF_Semestre_SINU','DescRF_ciclo','DescRF_Genero', 'DescRF_Jornada', 'DescRF_Periodo_Ult_Pago','DescRF_Sede',                      
'DescRF_Regional','DescRF_Programa','DescRF_tipo_estudio',
'DescRF_Tipo_Alumno','DescRF_Nuevo','DescAM_MATERIAS_APROBADAS','DescAM_Porcentaje_aprobacion',
'EEpromedio','EE_OTRA_DISCAPACIDAD','EE_SISTEMA_SALUD',                       
'EE_ESTRATO_ACTUALIZADO','EE_EMPRESA','NOT_PERIODO','ESTADO_PAGO','EstadoMoodle',
'Horario','VALOR_FINANCIACION','ICETEX','FINCOMERCIO','CREDITY','JAC',
'CONVENIOV','Total_Tickets','Trabaja','diferencia_dias_moodle_u','DescAM_MATERIAS_INSCRITAS']


# 4. Codificación de variables categóricas (Label Encoding)
# Guardamos los encoders por si necesitamos invertir la transformación luego
df_procesado = df_final.copy()
encoders = {}

cat_cols = df_procesado[todas_las_variables].select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    le = LabelEncoder()
    # Convertimos a string para manejar posibles nulos como categoría
    df_procesado[col] = le.fit_transform(df_procesado[col].astype(str))
    encoders[col] = le

# 5. Definir X (Variables) y y (Objetivo)
X = df_procesado[todas_las_variables]
y = df_procesado['DescRF_Status']

# 6. División Train/Test (Para validar que los modelos funcionan)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Datos procesados y listos para entrenar.")

Datos procesados y listos para entrenar.


In [88]:
import os
X_train.head(1)

,Identificacion,DescRF_Status,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_Genero,DescRF_Jornada,DescRF_Periodo_Ult_Pago,DescRF_Sede,DescRF_Regional,DescRF_Programa,DescRF_tipo_estudio,DescRF_Tipo_Alumno,DescRF_Nuevo,DescAM_MATERIAS_APROBADAS,DescAM_Porcentaje_aprobacion,EEpromedio,EE_OTRA_DISCAPACIDAD,EE_SISTEMA_SALUD,EE_ESTRATO_ACTUALIZADO,EE_EMPRESA,NOT_PERIODO,ESTADO_PAGO,EstadoMoodle,Horario,VALOR_FINANCIACION,ICETEX,FINCOMERCIO,CREDITY,JAC,CONVENIOV,Total_Tickets,Trabaja,diferencia_dias_moodle_u,DescAM_MATERIAS_INSCRITAS
311985,29808,0,1,2,1,1,1,2,277,2,2,20,0,2,1,0,0.0,0,0,0,0,0,0.1,0,3,1,668522.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,8


## paso 2

In [89]:
import os
# -----------------------------------------------------------------------------
# 1. DEFINICIÓN DE VARIABLES NUEVAS QUE ENTRAN CADA SEMANA (Según tu lista)
# -----------------------------------------------------------------------------


#Semana 0 
nuevas_sem_0 = ['DescAM_MATERIAS_INSCRITAS','DescAM_MATERIAS_APROBADAS','DescAM_Porcentaje_aprobacion',
'EE_OTRA_DISCAPACIDAD','EE_SISTEMA_SALUD','EE_ESTRATO_ACTUALIZADO','EE_EMPRESA',               
'ESTADO_PAGO','EstadoMoodle','Horario','VALOR_FINANCIACION','ICETEX',
'FINCOMERCIO','CREDITY','JAC','CONVENIOV','Total_Tickets','Trabaja',
'diferencia_dias_moodle_u']

# Semana 1: Datos estáticos, financieros iniciales y demográficos
nuevas_sem_1 = ['DescRF_Tipo_estado_alumno','DescRF_Modalidad',
'DescRF_Semestre_SINU','DescRF_ciclo','DescRF_Genero',
'DescRF_Jornada', 'DescRF_Periodo_Ult_Pago', 'DescRF_Sede','DescRF_Regional',
'DescRF_Programa', 'DescRF_tipo_estudio','DescRF_Tipo_Alumno',
'DescRF_Nuevo'
]

# Semana 2: Entran Fuente, Reglas Moodle, Ciclo Cod, Créditos y Tickets
nuevas_sem_2 = [
]

# Semana 3: Entra Acceso real a Moodle y Deuda
nuevas_sem_3 = [    
]

# Semana 4: Entra Estado de Grado y Rango Aprobación
nuevas_sem_4 = ['EEpromedio']

# Semana 5: Entra Estado de Grado y Rango Aprobación
nuevas_sem_5 = []

# Semana 6: Entra el Promedio (EE)
nuevas_sem_6 = [
]

# Semana 7: Entra Pérdida Académica
nuevas_sem_7 = [    
]

# Semana 8: Entran Resultados Finales/Parciales
nuevas_sem_8 = [             
]

# Semana 9: Entran Resultados Finales/Parciales
nuevas_sem_9 = [
]

# Semana 10: Entran Resultados Finales/Parciales
nuevas_sem_10 = [
]

# Semana 11: Entran Resultados Finales/Parciales
nuevas_sem_11 = [
]

# Semana 12: Entran Resultados Finales/Parciales
nuevas_sem_12 = []

# Semana 13: Entran Resultados Finales/Parciales
nuevas_sem_13 = [
]

# Semana 14: Entran Resultados Finales/Parciales
nuevas_sem_14 = []

# Semana 15: Entran Resultados Finales/Parciales
nuevas_sem_15 = ['NOT_PERIODO']

# Semana 16: Entran Resultados Finales/Parciales
nuevas_sem_16 = []

# NOTA SOBRE SEMANA 15 y 16:
# 'DescRF_EX_Periodo_Siguiente_Normal' (Sem 15) y 'DescRF_Tipo_estado_alumno' (Sem 16)
# suelen ser variables de "autopsia" (cuando ya desertó). 
# Si quieres predecir, generalmente paramos en la semana 8 o 10. 
# Pero si las necesitas, agrégalas aquí abajo siguiendo la lógica.

# -----------------------------------------------------------------------------
# 2. CREACIÓN DE ACUMULADOS (El modelo de la semana 4 debe saber lo de la 1, 2 y 3)
# -----------------------------------------------------------------------------

# Diccionario maestro de variables por modelo
vars_acumuladas = {
    "Modelo_Semana_0": nuevas_sem_0,
    "Modelo_Semana_1": nuevas_sem_0 + nuevas_sem_1,
    "Modelo_Semana_2": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2,
    "Modelo_Semana_3": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3,
    "Modelo_Semana_4": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4,
    "Modelo_Semana_5": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5,
    "Modelo_Semana_6": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6,
    "Modelo_Semana_7": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7,
    "Modelo_Semana_8": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8,
    "Modelo_Semana_9": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9,
    "Modelo_Semana_10": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9 + nuevas_sem_10,
    "Modelo_Semana_11": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9 + nuevas_sem_10 + nuevas_sem_11,
    "Modelo_Semana_12": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9 + nuevas_sem_10 + nuevas_sem_11 + nuevas_sem_12,
    "Modelo_Semana_13": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9 + nuevas_sem_10 + nuevas_sem_11 + nuevas_sem_12 + nuevas_sem_13,
    "Modelo_Semana_14": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9 + nuevas_sem_10 + nuevas_sem_11 + nuevas_sem_12 + nuevas_sem_13 + nuevas_sem_14,
    "Modelo_Semana_15": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9 + nuevas_sem_10 + nuevas_sem_11 + nuevas_sem_12 + nuevas_sem_13 + nuevas_sem_14 + nuevas_sem_15,   
    "Modelo_Semana_16": nuevas_sem_0 + nuevas_sem_1 + nuevas_sem_2 + nuevas_sem_3 + nuevas_sem_4 + nuevas_sem_5 + nuevas_sem_6 + nuevas_sem_7 + nuevas_sem_8 + nuevas_sem_9 + nuevas_sem_10 + nuevas_sem_11 + nuevas_sem_12 + nuevas_sem_13 + nuevas_sem_14 + nuevas_sem_15 + nuevas_sem_16
}

print("Variables organizadas acumulativamente.")
print(f"Variables en Semana 1: {len(vars_acumuladas['Modelo_Semana_1'])}")
print(f"Variables en Semana 8: {len(vars_acumuladas['Modelo_Semana_8'])}")

Variables organizadas acumulativamente.
Variables en Semana 1: 32
Variables en Semana 8: 33


In [90]:
import os
X_train.head(1)

,Identificacion,DescRF_Status,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_Genero,DescRF_Jornada,DescRF_Periodo_Ult_Pago,DescRF_Sede,DescRF_Regional,DescRF_Programa,DescRF_tipo_estudio,DescRF_Tipo_Alumno,DescRF_Nuevo,DescAM_MATERIAS_APROBADAS,DescAM_Porcentaje_aprobacion,EEpromedio,EE_OTRA_DISCAPACIDAD,EE_SISTEMA_SALUD,EE_ESTRATO_ACTUALIZADO,EE_EMPRESA,NOT_PERIODO,ESTADO_PAGO,EstadoMoodle,Horario,VALOR_FINANCIACION,ICETEX,FINCOMERCIO,CREDITY,JAC,CONVENIOV,Total_Tickets,Trabaja,diferencia_dias_moodle_u,DescAM_MATERIAS_INSCRITAS
311985,29808,0,1,2,1,1,1,2,277,2,2,20,0,2,1,0,0.0,0,0,0,0,0,0.1,0,3,1,668522.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,8


In [91]:
import os
vars_acumuladas['Modelo_Semana_1']

['DescAM_MATERIAS_INSCRITAS',
 'DescAM_MATERIAS_APROBADAS',
 'DescAM_Porcentaje_aprobacion',
 'EE_OTRA_DISCAPACIDAD',
 'EE_SISTEMA_SALUD',
 'EE_ESTRATO_ACTUALIZADO',
 'EE_EMPRESA',
 'ESTADO_PAGO',
 'EstadoMoodle',
 'Horario',
 'VALOR_FINANCIACION',
 'ICETEX',
 'FINCOMERCIO',
 'CREDITY',
 'JAC',
 'CONVENIOV',
 'Total_Tickets',
 'Trabaja',
 'diferencia_dias_moodle_u',
 'DescRF_Tipo_estado_alumno',
 'DescRF_Modalidad',
 'DescRF_Semestre_SINU',
 'DescRF_ciclo',
 'DescRF_Genero',
 'DescRF_Jornada',
 'DescRF_Periodo_Ult_Pago',
 'DescRF_Sede',
 'DescRF_Regional',
 'DescRF_Programa',
 'DescRF_tipo_estudio',
 'DescRF_Tipo_Alumno',
 'DescRF_Nuevo']

## paso 3

In [92]:
import os
import joblib
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

# ---------------------------------------------------------
# 3. ENTRENAMIENTO DE MODELOS (Iteración Automática)
# ---------------------------------------------------------

# Verificación de seguridad: Asegurarnos de que el paso 2 se corrió
if 'vars_acumuladas' not in locals():
    print("❌ ERROR: No se encuentra 'vars_acumuladas'. Por favor ejecuta el código del Paso 2 primero.")
else:
    print(f"✅ Iniciando entrenamiento de {len(vars_acumuladas)} modelos semanales...\n")
    
    # Diccionario para guardar los resultados en memoria (opcional, por si quieres verlos luego)
    resultados_entrenamiento = []

    for nombre_modelo, columnas_etapa in vars_acumuladas.items():
        print(f"🔄 Procesando: {nombre_modelo}...")
        print(f"   Variables disponibles: {len(columnas_etapa)}")
        
        # 1. Filtrar los datos para usar SOLO las columnas de esta semana
        # Si intentamos usar columnas que no están en 'columnas_etapa', el modelo fallaría
        try:
            X_train_etapa = X_train[columnas_etapa]
            X_test_etapa = X_test[columnas_etapa]
        except KeyError as e:
            print(f"   ❌ Error: Alguna variable definida en el Paso 2 no existe en el dataset. Revisa: {e}")
            continue

        # 2. Configuración del modelo LightGBM
        # Usamos parámetros robustos para evitar sobreajuste
        clf = LGBMClassifier(
            n_estimators=200,
            learning_rate=0.01,
            num_leaves=127,
            max_depth=20,
            subsample=0.8,
            colsample_bytree=0.6,
            random_state=42,
            n_jobs=-1,
            verbose=-1 # Para que no llene la pantalla de mensajes técnicos
        )
        
        # 3. Entrenar
        clf.fit(X_train_etapa, y_train)
        
        # 4. Guardar el modelo en un archivo .pkl
        nombre_archivo = f"{nombre_modelo}.pkl"
        joblib.dump(clf, nombre_archivo)
        
        # 5. Validar calidad (ROC-AUC)
        # Calculamos qué tan bueno es este modelo con los datos de prueba
        probs_test = clf.predict_proba(X_test_etapa)[:, 1]
        auc = roc_auc_score(y_test, probs_test)
        
        print(f"   💾 Guardado como: {nombre_archivo}")
        print(f"   📊 Precisión (ROC-AUC): {auc:.4f}")
        print("-" * 30)
        
        resultados_entrenamiento.append({
            "Semana": nombre_modelo,
            "Variables": len(columnas_etapa),
            "AUC": auc
        })

    print("\n🚀 ¡Entrenamiento finalizado! Todos los modelos están listos para usar.")
    
    # Mostrar resumen final
    resumen_df = pd.DataFrame(resultados_entrenamiento)
    display(resumen_df)

✅ Iniciando entrenamiento de 17 modelos semanales...

🔄 Procesando: Modelo_Semana_0...
   Variables disponibles: 19
   💾 Guardado como: Modelo_Semana_0.pkl
   📊 Precisión (ROC-AUC): 0.9048
------------------------------
🔄 Procesando: Modelo_Semana_1...
   Variables disponibles: 32
   💾 Guardado como: Modelo_Semana_1.pkl
   📊 Precisión (ROC-AUC): 0.9545
------------------------------
🔄 Procesando: Modelo_Semana_2...
   Variables disponibles: 32
   💾 Guardado como: Modelo_Semana_2.pkl
   📊 Precisión (ROC-AUC): 0.9545
------------------------------
🔄 Procesando: Modelo_Semana_3...
   Variables disponibles: 32
   💾 Guardado como: Modelo_Semana_3.pkl
   📊 Precisión (ROC-AUC): 0.9545
------------------------------
🔄 Procesando: Modelo_Semana_4...
   Variables disponibles: 33
   💾 Guardado como: Modelo_Semana_4.pkl
   📊 Precisión (ROC-AUC): 0.9545
------------------------------
🔄 Procesando: Modelo_Semana_5...
   Variables disponibles: 33
   💾 Guardado como: Modelo_Semana_5.pkl
   📊 Precisión

,Semana,Variables,AUC
0,Modelo_Semana_0,19,0.904826
1,Modelo_Semana_1,32,0.954475
2,Modelo_Semana_2,32,0.954475
3,Modelo_Semana_3,32,0.954475
4,Modelo_Semana_4,33,0.954475
5,Modelo_Semana_5,33,0.954475
6,Modelo_Semana_6,33,0.954475
7,Modelo_Semana_7,33,0.954475
8,Modelo_Semana_8,33,0.954475
9,Modelo_Semana_9,33,0.954475


## paso 4

In [96]:
import os
import pandas as pd
import numpy as np
import shap
import joblib

def generar_matriz_shap_semanal(df_datos, semana_actual):
    """
    Entrada: 
        - df_datos: DataFrame con los datos de los estudiantes (debe incluir 'DescRF_Identificacion').
                    NOTA: Las variables ya deben estar codificadas numéricamente (como en X_test).
        - semana_actual: Entero (1 a 16).
    
    Salida: 
        - DataFrame con ID, valores SHAP de cada variable y Probabilidad de Deserción.
    """
    
    # ---------------------------------------------------------
    # 1. SELECCIÓN DEL MODELO SEGÚN LA SEMANA
    # ---------------------------------------------------------
    # Usamos la lógica de "último modelo disponible"
    
    nombre_modelo = ""
    
    if semana_actual == 0:
        nombre_modelo = "Modelo_Semana_0"
    elif semana_actual == 1:
        nombre_modelo = "Modelo_Semana_1"
    elif semana_actual == 2:
        nombre_modelo = "Modelo_Semana_2"
    elif semana_actual == 3:
        nombre_modelo = "Modelo_Semana_3"
    elif semana_actual == 4:
        nombre_modelo = "Modelo_Semana_4"         
    elif semana_actual == 5: 
        nombre_modelo = "Modelo_Semana_5"
    elif semana_actual == 6:
        nombre_modelo = "Modelo_Semana_6"
    elif semana_actual == 7:
        nombre_modelo = "Modelo_Semana_7"
    elif semana_actual == 8: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_8"
    elif semana_actual == 9: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_9"
    elif semana_actual == 10: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_10"   
    elif semana_actual == 11: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_11"
    elif semana_actual == 12: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_12"
    elif semana_actual == 13: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_13"
    elif semana_actual == 14: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_14"
    elif semana_actual == 15: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_15"
    elif semana_actual == 16: 
        # De la 8 en adelante usamos el modelo completo
        nombre_modelo = "Modelo_Semana_16"
    else:
        return f"Error: La semana {semana_actual} no es válida."

    # Recuperamos las columnas exactas que usa ese modelo desde el diccionario del Paso 2
    try:
        cols_modelo = vars_acumuladas[nombre_modelo]
    except KeyError:
        return "Error: No se encontró la definición de variables. Asegúrate de haber corrido el Paso 2."

    print(f"--> Analizando Semana {semana_actual} usando: {nombre_modelo}")
    print(f"    Variables analizadas: {len(cols_modelo)}")

    # ---------------------------------------------------------
    # 2. CARGA DEL MODELO Y PREPARACIÓN DE DATOS
    # ---------------------------------------------------------
    try:
        modelo = joblib.load(f"{nombre_modelo}.pkl")
    except FileNotFoundError:
        return f"Error: No existe el archivo {nombre_modelo}.pkl. Ejecuta el Paso 3 primero."

    # Aseguramos que el ID exista
    if 'Identificacion' not in df_datos.columns:
        return "Error: El DataFrame entregado no tiene la columna 'Identificacion'."

    ids = df_datos['Identificacion'].astype(str).values
    
    # Filtramos las columnas (X) asegurando el orden correcto
    try:
        X_pred = df_datos[cols_modelo]
    except KeyError as e:
        return f"Error: Faltan columnas en los datos entregados para esta semana: {e}"

    # ---------------------------------------------------------
    # 3. PREDICCIÓN Y SHAP
    # ---------------------------------------------------------
    
    # Probabilidad (0-100)
    probs = modelo.predict_proba(X_pred)[:, 1] * 100
    
    # SHAP
    print("    Calculando atribución de riesgo (SHAP)...")
    explainer = shap.TreeExplainer(modelo)
    shap_values = explainer.shap_values(X_pred)
    
    # Manejo de formato de salida de shap (LightGBM a veces devuelve lista)
    if isinstance(shap_values, list):
        shap_values = shap_values[1]

    # ---------------------------------------------------------
    # 4. CONSTRUCCIÓN DE LA TABLA FINAL
    # ---------------------------------------------------------
    
    # Creamos DF con los valores SHAP
    df_shap = pd.DataFrame(shap_values, columns=cols_modelo)
    
    # Redondeo a 2 decimales para limpieza visual
    df_shap = df_shap.round(2)
    
    # Insertamos ID al inicio y Probabilidad al final
    df_shap.insert(0, 'Identificacion', ids)
    df_shap['Prob_Desercion'] = np.round(probs, 2)
    
    return df_shap

print("¡Función 'generar_matriz_shap_semanal' creada exitosamente!")

¡Función 'generar_matriz_shap_semanal' creada exitosamente!


## paso 5

In [97]:
import os
# 1. Preparar una muestra de prueba con ID
indices_muestra = X_test.index
muestra_datos = X_test.copy()
# Recuperamos el ID original
muestra_datos['Identificacion'] = df_final.loc[indices_muestra, 'Identificacion']

# 2. PROBAR LA FUNCIÓN (Simulando Semana 4)
df_resultado = generar_matriz_shap_semanal(muestra_datos, semana_actual=4)

if isinstance(df_resultado, pd.DataFrame):
    print("\n--- EJEMPLO DE RESULTADO (Primeras 5 filas) ---")
    display(df_resultado.head())
    
    # Opcional: Guardar a Excel
    # df_resultado.to_excel("Reporte_Semana_4.xlsx", index=False)
else:
    print(df_resultado) # Imprimir error si ocurre

--> Analizando Semana 4 usando: Modelo_Semana_4
    Variables analizadas: 33
    Calculando atribución de riesgo (SHAP)...


C:\Users\juan_marin\Documents\CUN\.venv\Lib\site-packages\shap\explainers\_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(



--- EJEMPLO DE RESULTADO (Primeras 5 filas) ---


,Identificacion,DescAM_MATERIAS_INSCRITAS,DescAM_MATERIAS_APROBADAS,DescAM_Porcentaje_aprobacion,EE_OTRA_DISCAPACIDAD,EE_SISTEMA_SALUD,EE_ESTRATO_ACTUALIZADO,EE_EMPRESA,ESTADO_PAGO,EstadoMoodle,Horario,VALOR_FINANCIACION,ICETEX,FINCOMERCIO,CREDITY,JAC,CONVENIOV,Total_Tickets,Trabaja,diferencia_dias_moodle_u,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_Genero,DescRF_Jornada,DescRF_Periodo_Ult_Pago,DescRF_Sede,DescRF_Regional,DescRF_Programa,DescRF_tipo_estudio,DescRF_Tipo_Alumno,DescRF_Nuevo,EEpromedio,Prob_Desercion
0,1054856586,0.02,0.57,0.99,0.0,0.0,0.0,0.0,-0.02,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.03,-0.01,0.0,0.0,-0.05,0.09,-0.01,0.11,0.04,-0.00,-0.01,0.16,-0.02,-0.0,0.03,0.0,0.05,0.04,0.0,87.52
1,52014592,0.02,-0.64,-0.90,0.0,0.0,0.0,0.0,-0.10,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.05,-0.02,0.0,0.0,-0.25,0.05,-0.02,0.07,0.02,0.00,-0.05,0.12,-0.02,-0.0,-0.01,0.0,0.02,0.02,0.0,15.19
2,45507706,-0.04,0.31,0.24,0.0,0.0,0.0,0.0,-0.00,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.05,-0.02,0.0,0.0,-0.01,-0.19,-0.10,-0.30,-0.49,0.01,0.01,0.04,-0.01,-0.0,-0.03,0.0,-0.21,-0.09,0.0,28.54
3,1073525995,-0.02,-0.61,-0.86,0.0,0.0,0.0,0.0,-0.10,-0.0,-0.0,-0.01,0.0,-0.0,-0.0,-0.09,-0.02,0.0,0.0,-0.21,0.11,-0.00,-0.00,-0.05,-0.00,-0.03,0.11,-0.02,-0.0,-0.01,0.0,0.05,0.03,0.0,15.07
4,1033096013,0.02,-0.62,-0.88,0.0,0.0,0.0,0.0,-0.07,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.04,-0.01,0.0,0.0,-0.17,0.05,0.13,0.06,0.02,-0.00,-0.06,0.11,-0.04,-0.0,-0.01,0.0,0.03,0.02,0.0,19.05


In [98]:
import os
df_resultado.head()

,Identificacion,DescAM_MATERIAS_INSCRITAS,DescAM_MATERIAS_APROBADAS,DescAM_Porcentaje_aprobacion,EE_OTRA_DISCAPACIDAD,EE_SISTEMA_SALUD,EE_ESTRATO_ACTUALIZADO,EE_EMPRESA,ESTADO_PAGO,EstadoMoodle,Horario,VALOR_FINANCIACION,ICETEX,FINCOMERCIO,CREDITY,JAC,CONVENIOV,Total_Tickets,Trabaja,diferencia_dias_moodle_u,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_Genero,DescRF_Jornada,DescRF_Periodo_Ult_Pago,DescRF_Sede,DescRF_Regional,DescRF_Programa,DescRF_tipo_estudio,DescRF_Tipo_Alumno,DescRF_Nuevo,EEpromedio,Prob_Desercion
0,1054856586,0.02,0.57,0.99,0.0,0.0,0.0,0.0,-0.02,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.03,-0.01,0.0,0.0,-0.05,0.09,-0.01,0.11,0.04,-0.00,-0.01,0.16,-0.02,-0.0,0.03,0.0,0.05,0.04,0.0,87.52
1,52014592,0.02,-0.64,-0.90,0.0,0.0,0.0,0.0,-0.10,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.05,-0.02,0.0,0.0,-0.25,0.05,-0.02,0.07,0.02,0.00,-0.05,0.12,-0.02,-0.0,-0.01,0.0,0.02,0.02,0.0,15.19
2,45507706,-0.04,0.31,0.24,0.0,0.0,0.0,0.0,-0.00,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.05,-0.02,0.0,0.0,-0.01,-0.19,-0.10,-0.30,-0.49,0.01,0.01,0.04,-0.01,-0.0,-0.03,0.0,-0.21,-0.09,0.0,28.54
3,1073525995,-0.02,-0.61,-0.86,0.0,0.0,0.0,0.0,-0.10,-0.0,-0.0,-0.01,0.0,-0.0,-0.0,-0.09,-0.02,0.0,0.0,-0.21,0.11,-0.00,-0.00,-0.05,-0.00,-0.03,0.11,-0.02,-0.0,-0.01,0.0,0.05,0.03,0.0,15.07
4,1033096013,0.02,-0.62,-0.88,0.0,0.0,0.0,0.0,-0.07,-0.0,-0.0,-0.00,0.0,-0.0,-0.0,-0.04,-0.01,0.0,0.0,-0.17,0.05,0.13,0.06,0.02,-0.00,-0.06,0.11,-0.04,-0.0,-0.01,0.0,0.03,0.02,0.0,19.05


In [102]:
import os
df_25V04 = df

In [103]:
import os
df_25V04.head(1)

,Identificacion,Periodo,DescRF_Status,DescRF_Tipo_estado_alumno,DescRF_Modalidad,DescRF_Semestre_SINU,DescRF_ciclo,DescRF_SEMESTRE_MEN,DescRF_Genero,DescRF_Unidad,DescRF_Jornada,DescRF_Periodo_Ult_Pago,DescRF_Sede,DescRF_Regional,DescRF_Programa,DescRF_Acceso_Moodle,DescRF_tipo_estudio,Id_Alumn_Programa,DescRF_Tipo_Alumno,DescRF_Nuevo,DescRF_orden,DescRF_Fecha_Liquidacion,DescRF_Fecha_Nacimiento,DescRF_ultimo_curso,DescAM_MATERIAS_INSCRITAS,DescAM_MATERIAS_APROBADAS,DescAM_Porcentaje_aprobacion,EEpromedio,EE_UNIDADNEGOCIO,EE_DEPARTAMENTO_REGIONAL,EE_OTRA_DISCAPACIDAD,EE_SISTEMA_SALUD,EE_ESTRATO_ACTUALIZADO,EE_EMPRESA,COD_MATERIA,DEF_HISTORIA,NOT_PERIODO,ESTADO_PAGO,EstadoMoodle,Ultimo_Acc_Mood,Correo,Horario,VALOR_FINANCIACION,RECIBOS_CAJA,ICETEX,FINCOMERCIO,CREDITY,JAC,CONVENIOV,Total_Tickets,Edad,Trabaja,diferencia_dias_moodle_u,NA_Calificaciones,NA_RECIBOS,NA_ICETEX,NA_FINCOMERCIO,NA_CREDITY,NA_Total_Tickets
0,1000004469,24V02,1,NUEVO,Virtual,8,PROFESIONAL,1,M,PROFESIONAL EN INGENIERIA DE SISTEMAS,JORNADA ÚNICA,24I02,BOGOTA CENTRO,Regional Bogotá,INGENIERIA DE SISTEMAS,Sin Moodle,PREGRADO,1152303,Nuevo,NUEVO,2451252,2024-03-18,2003-04-11,SI,6,3,50.0,None,None,None,None,None,None,None,TIE05,4.5,4.5,NO,ACTIVO,NaT,NO,SI,NaN,1539000.0,0.0,0.0,0.0,0.0,0.0,0.0,22,NO TRABAJA,NaN,False,False,False,False,False,True


In [106]:
import os
df.Periodo.value_counts(dropna=False).head()

Periodo
25V02    172595
25V01    167687
2025A     67884
24V04     58930
24V01     51756
Name: count, dtype: int64

In [104]:
import os
df_25V04 = df_25V04[df_25V04['Periodo']=='25V04']

In [105]:
import os
df_25V04.shape

(0, 59)

## Código para probar con datos nuevos

In [ ]:
import os


In [ ]:
import os
# 1. Cargar los datos nuevos
# Asegúrate de que este archivo tenga las columnas hasta la semana 4 llenas
df_nuevos = pd.read_excel("estudiantes_nuevos.xlsx") 

# 2. PREPROCESAMIENTO (Convertir texto a números)
# Esto es obligatorio. Usamos el diccionario 'encoders' que creamos en el Paso 1.
# Si la variable es numérica (ej: Edad), no se toca. Si es texto, se transforma.

df_nuevos_procesado = df_nuevos.copy()

# Limpieza de ID igual que en el entrenamiento
df_nuevos_procesado['DescRF_Identificacion'] = df_nuevos_procesado['DescRF_Identificacion'].astype(str).str.strip()

# Aplicamos la misma transformación que al entrenar
for col, encoder in encoders.items():
    if col in df_nuevos_procesado.columns:
        # Manejo de error por si aparece una categoría nueva que el modelo no conoce
        # Ej: Aparece Regional "Japon" y el modelo solo conoce las de Colombia.
        # Lo convertimos a texto, y si no existe en el encoder, asignamos un valor por defecto o fallamos controlado.
        
        # Opción rápida: Usar map y llenar nulos con -1 o moda
        series_col = df_nuevos_procesado[col].astype(str)
        
        # Truco para aplicar el encoder de forma segura
        # (Si hay una categoría nueva desconocida, esto podría dar error, 
        # en producción se usan técnicas más avanzadas, pero para prueba esto sirve)
        try:
            df_nuevos_procesado[col] = encoder.transform(series_col)
        except ValueError as e:
            # Si hay una categoría nueva, imprimimos advertencia
            print(f"⚠️ Advertencia en columna '{col}': Hay datos que el modelo no conoce. {e}")
            # Una solución parche es re-entrenar el encoder o asignar la clase mayoritaria,
            # pero para esta prueba, intentemos que los datos coincidan con lo conocido.

# 3. EJECUTAR LA PREDICCIÓN PARA SEMANA 4
# Aquí es donde no importa si las columnas de semana 8 son NaN, porque la función las ignora.
df_resultado_prueba = generar_matriz_shap_semanal(df_nuevos_procesado, semana_actual=4)

# 4. VER RESULTADOS
if isinstance(df_resultado_prueba, pd.DataFrame):
    print("\n--- RESULTADOS PARA ESTUDIANTES NUEVOS (Semana 4) ---")
    display(df_resultado_prueba)
    
    # Exportar
    df_resultado_prueba.to_excel("Prediccion_Estudiantes_Nuevos_Sem4.xlsx", index=False)
else:
    print(df_resultado_prueba)